## Link Prediction
https://docs.dgl.ai/tutorials/blitz/4_link_predict.html#sphx-glr-tutorials-blitz-4-link-predict-py

In [1]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
import itertools
import numpy as np
import scipy.sparse as sp

In [2]:
import networkx as nx

## Overview of Link Prediction with GNN

This tutorial formulates the link prediction problem as a binary classification problem as follows:
- Treat the edges in the graph as *positive examples*.
- Sample a number of non-existent edges (i.e. node paris with no edges between them) as *negative* examples
- Divide the positive examples and negative examples into a training set and a test set.
- Evaluate the model with any binary classification metric such as Area Under Curve (AUC).

<div class="alert alert-info">
<b>NOTE:</b> 
    The practice comes from SEAL, although the model here does not use their idea of node labeling.
</div>

In some domains such as large-scale recommender systems or information retrieval, you may favor metrics that emphasize good performance or top-K predictions. In these cases you may want to consider other metrics such as mean average precision, and use other negative sampling methods.

## Loading graph and features

In [3]:
##
dataset = dgl.data.CoraGraphDataset()
dataset

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


Dataset("cora_v2", num_graphs=1, save_path=/home/chuck/.dgl/cora_v2)

In [4]:
try:
    dataset[1]
except AssertionError as e:
    print(e)

This dataset has only one graph


In [5]:
##
g = dataset[0]
g

Graph(num_nodes=2708, num_edges=10556,
      ndata_schemes={'feat': Scheme(shape=(1433,), dtype=torch.float32), 'label': Scheme(shape=(), dtype=torch.int64), 'test_mask': Scheme(shape=(), dtype=torch.bool), 'val_mask': Scheme(shape=(), dtype=torch.bool), 'train_mask': Scheme(shape=(), dtype=torch.bool)}
      edata_schemes={'__orig__': Scheme(shape=(), dtype=torch.int64)})

In [25]:
g.__dict__

{'_graph': <dgl.heterograph_index.HeteroGraphIndex at 0x7fe1c45bc0c0>,
 '_canonical_etypes': [('_N', '_E', '_N')],
 '_batch_num_nodes': None,
 '_batch_num_edges': None,
 '_ntypes': ['_N'],
 '_is_unibipartite': False,
 '_srctypes_invmap': {'_N': 0},
 '_dsttypes_invmap': {'_N': 0},
 '_etypes': ['_E'],
 '_etype2canonical': {'_E': ('_N', '_E', '_N')},
 '_etypes_invmap': {('_N', '_E', '_N'): 0},
 '_node_frames': [{'feat': tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0526, 0.0000]]), 'label': tensor([4, 4, 4,  ..., 4, 3, 3]), 'test_mask': tensor([ True,  True, False,  ..., False, False, False]), 'val_mask': tensor([False, False,  True,  ..., False, 

In [38]:
print(f'is_unibipartite: {g.is_unibipartite}')
print(f'is_multigraph: {g.is_multigraph}')
print(f'is_homogeneous: {g.is_homogeneous}')


print(f'g.ntypes: {g.ntypes}')
print(f'g.etypes: {g.etypes}')

print(f'canonical_etypes: {g.canonical_etypes}')
print(f'srctypes: {g.srctypes}')
print(f'dsttypes: {g.dsttypes}')
print(f'metagraph: {g.metagraph()}')

is_unibipartite: False
is_multigraph: False
is_homogeneous: True
g.ntypes: ['_N']
g.etypes: ['_E']
canonical_etypes: [('_N', '_E', '_N')]
srctypes: ['_N']
dsttypes: ['_N']
metagraph: MultiDiGraph with 1 nodes and 1 edges


In [44]:
g.node_attr_schemes()

{'feat': Scheme(shape=(1433,), dtype=torch.float32),
 'label': Scheme(shape=(), dtype=torch.int64),
 'test_mask': Scheme(shape=(), dtype=torch.bool),
 'val_mask': Scheme(shape=(), dtype=torch.bool),
 'train_mask': Scheme(shape=(), dtype=torch.bool)}

In [45]:
## ndata
print("--- ndata ---")
for k in g.ndata.keys():
    print(k)
    if torch.is_tensor(g.ndata[k]):
        print(g.ndata[k].shape)
    else:
        print(g.ndata[k])
    print()
    
## edata
print("--- edata ---")
for k in g.edata.keys():
    print(k)
    if torch.is_tensor(g.edata[k]):
        print(g.edata[k].shape)
    else:
        print(g.edata[k])
    print()

--- ndata ---
feat
torch.Size([2708, 1433])

label
torch.Size([2708])

test_mask
torch.Size([2708])

val_mask
torch.Size([2708])

train_mask
torch.Size([2708])

--- edata ---
__orig__
torch.Size([10556])



In [70]:
print(f'nodes: {g.nodes()}')

nodes: tensor([   0,    1,    2,  ..., 2705, 2706, 2707])


In [61]:
print(f'edges: {g.edges()}')

edges: (tensor([   3,    5,    6,  ..., 2704, 2707, 2706]), tensor([   0,    0,    0,  ..., 2705, 2706, 2707]))


In [66]:
g.edge_ids(5, 0)

1

In [74]:
print(f'source nodes: {g.srcnodes()}, {g.number_of_src_nodes()}')

source nodes: tensor([   0,    1,    2,  ..., 2705, 2706, 2707]), 2708


In [75]:
print(f'destination nodes: {g.dstnodes()}, {g.number_of_dst_nodes()}')

destination nodes: tensor([   0,    1,    2,  ..., 2705, 2706, 2707]), 2708


## Prepare training and testing sets

This tutorial randomly picks 10% of the edges for positive examples in the test set and leaves the rest for the training set. It then samples the same number of edges for negative examples in both sets.

In [71]:
# separate (src, dst) pairs of each edge
u, v = g.edges()

In [184]:
len(u), len(v)

(10556, 10556)

In [78]:
# create edge indices and shuffle them
eids = np.arange(g.number_of_edges())
eids = np.random.permutation(eids)
eids

array([8382, 8684, 2378, ..., 3594, 6782, 6159])

In [83]:
# split into train/test sets
test_size = int(len(eids) * 0.1)
test_eids = eids[:test_size]
test_pos_u, test_pos_v = u[test_eids], v[test_eids]

train_size = g.number_of_edges() - test_size
train_eids = eids[test_size:]
train_pos_u, train_pos_v = u[train_eids], v[train_eids]

#### Aside: finding negative edges
- Here we try to understand the method used in the tutorial to find the negative edges. 
- It seems to assume there are no self-loops.

In [94]:
# generate a random adjacency matrix
n = 4
a = np.random.randint(0, 2, size=(n, n))
a

array([[1, 1, 0, 1],
       [1, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 1]])

In [145]:
# remove self-loops
for i in range(n):
    a[i, i] = 0

a

array([[0, 1, 0, 1],
       [1, 0, 1, 0],
       [0, 0, 0, 0],
       [0, 0, 1, 0]])

In [161]:
# want to determine where the negative edges are like this, but remove self-loops
nm = np.where(a==0)
print(nm)

(array([0, 0, 1, 1, 2, 2, 2, 2, 3, 3, 3]), array([0, 2, 1, 3, 0, 1, 2, 3, 0, 1, 3]))


In [162]:
# this is the form of g.adjacency_matrix() (our starting point)
am = np.where(a==1) 
print(am)

(array([0, 0, 1, 1, 3]), array([1, 3, 0, 2, 2]))


In [163]:
# create adjacency matrix
uu, vv = am
adjm = sp.coo_matrix((np.ones(len(uu)), (uu, vv))).todense()
print(adjm)

[[0. 1. 0. 1.]
 [1. 0. 1. 0.]
 [0. 0. 0. 0.]
 [0. 0. 1. 0.]]


In [164]:
# observe where adjacency has missing links; note the presence of the self-loops
adjm_sp = list(zip(*np.where(adjm == 0)))
adjm_sp

[(0, 0),
 (0, 2),
 (1, 1),
 (1, 3),
 (2, 0),
 (2, 1),
 (2, 2),
 (2, 3),
 (3, 0),
 (3, 1),
 (3, 3)]

In [151]:
# start with all ones, remove the adjacency matrix ones, and then remove the self-loops
negm = 1 - adjm - np.eye(n)
negm

matrix([[0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [1., 1., 0., 1.],
        [1., 1., 0., 0.]])

In [165]:
# this is our desired set of negative pairs
negm_sp = list(zip(*np.where(negm != 0)))
negm_sp

[(0, 2), (1, 3), (2, 0), (2, 1), (2, 3), (3, 0), (3, 1)]

In [166]:
# comparison to show the need for removing self-loops
print(set(negm_sp).difference(set(adjm_sp)))
print(set(adjm_sp).difference(set(negm_sp)))
print(set(negm_sp).symmetric_difference(set(adjm_sp)))

set()
{(1, 1), (3, 3), (2, 2), (0, 0)}
{(0, 0), (1, 1), (3, 3), (2, 2)}


#### Resume

In [181]:
# find all negative edges
adj = sp.coo_matrix((np.ones(len(u)), (u.numpy(), v.numpy())))

assert adj.shape[0] == g.number_of_nodes()  # LHS probably more reliable; RHS follows tutorial
adj_neg = 1 - adj.todense() - np.eye(g.number_of_nodes())
neg_u, neg_v = np.where(adj_neg != 0)

In [185]:
# the number of negative edges is much larger than the number of positive edges 
# because it represents every possible edge minus the positive edges
len(neg_u), len(neg_v)

(7320000, 7320000)

As noted above, we are going to sample a number of negative edges equal to the number of positive edges.
- assign an index to each element of `neg_u`, randomly permute them, and then keep a number equal to the number of positive edges
- Note: this is different than the tutorial which samples with replacement

In [244]:
# split negative edges into train/test
# neg_eids = np.random.choice(len(neg_u), g.number_of_edges())  # tutorial method
neg_eids = np.random.permutation(np.arange(len(eids)))[:g.number_of_edges()]
test_neg_u, test_neg_v = neg_u[neg_eids[:test_size]], neg_v[neg_eids[:test_size]]
train_neg_u, train_neg_v = neg_u[neg_eids[:train_size]], neg_v[neg_eids[:train_size]]

In [250]:
# verify we have the same number of positive and negative edges
assert len(neg_eids) == len(eids)
assert len(test_pos_u) == len(test_neg_u)

When training, we need to remove the edges in the test set from the original graph. This can be done with `dgl.remove_edges`.

<div class="alert alert-info">
    <b>Note</b>: <code>dgl.remove_edges</code> works by creating a subgraph from the original graph which results in a copy.
</div>   

In [254]:
## get train graph by removing test edges from the full graph
## reminder: `eids[:test_size]` are the indices that were used to sample from `u`, `v` to make the test set
train_g = dgl.remove_edges(g, eids[:test_size])

## Define a GraphSAGE model

This tutorial builds a model consisting of two GraphSAGE layers, each computes new node representations by averaging neighbor information. DGL provides `dgl.nn.SAGEConv` that conveniently creates a GraphSAGE layer.

In [255]:
from dgl.nn import SAGEConv

In [256]:
## build a two-layer GraphSAGE model
class GraphSAGE(nn.Module):
    def __init__(self, in_feats, h_feats):
        super().__init__()
        self.conv1 = SAGEConv(in_feats, h_feats, 'mean')
        self.conv2 = SAGEConv(h_feats, h_feats, 'mean')
        
    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h